#Extract
Authors: Gustavo FLEURY && Induraj RAMAMURTHY

Project: https://github.com/gustavofleury/Audit_Reports_NLP

In [0]:
!pip install selenium

In [0]:
!wget https://bitbucket.org/ariya/phantomjs/downloads/phantomjs-2.1.1-linux-x86_64.tar.bz2

In [0]:
!tar xvjf phantomjs-2.1.1-linux-x86_64.tar.bz2

In [0]:
!cp phantomjs-2.1.1-linux-x86_64/bin/phantomjs /usr/local/bin

In [0]:
#https://selenium-python.readthedocs.io/locating-elements.html
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException

import pandas as pd
import re
# import time

In [0]:
driver = webdriver.PhantomJS('phantomjs')
driver.implicitly_wait(30)

/usr/local/lib/python3.6/dist-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [0]:
#Verify if AJAX table are finished to load. (Combine with Wait.until)
def ajax_complete(driver):
    try:
        return 0 == driver.execute_script("return jQuery.active")
    except WebDriverException:
        pass

In [0]:
dfTotal= pd.DataFrame()
#All
URL = "https://auditoria.cgu.gov.br/"
fileName = "/content/drive/My Drive/Research/DataSets/DF_Links_Reports_ALL.pkl"        

# Auditoria Anual de Contas - AAC
# URL = "https://auditoria.cgu.gov.br/?draw=3&colunaOrdenacao=dataPublicacao&direcaoOrdenacao=DESC&tamanhoPagina=15&offset=0&titulo=&linhaAtuacao=1&de=&ate=&orgaos=&ujtcus=&estados=&municipios=&fefs=&palavraChave=#lista"
# fileName = "/content/drive/My Drive/Research/DataSets/DF_Links_Reports_AAC.pkl"

#Auditoria Gestao - AAG - DONE
# URL = "https://auditoria.cgu.gov.br/?draw=5&colunaOrdenacao=dataPublicacao&direcaoOrdenacao=DESC&tamanhoPagina=15&offset=0&titulo=&linhaAtuacao=3&de=&ate=&orgaos=&ujtcus=&estados=&municipios=&fefs=&palavraChave=#lista"
# fileName = "/content/drive/My Drive/Research/DataSets/DF_Links_Reports_AAG.pkl"

#Avaliação dos Resultados da Gestão - ARG
# URL = "https://auditoria.cgu.gov.br/?draw=2&colunaOrdenacao=dataPublicacao&direcaoOrdenacao=DESC&tamanhoPagina=15&offset=0&titulo=&linhaAtuacao=7&de=&ate=&orgaos=&ujtcus=&estados=&municipios=&fefs=&palavraChave=#lista"
# fileName = "/content/drive/My Drive/Research/DataSets/DF_Links_Reports_ARG.pkl"

driver.get(URL)
print('passed')
i=0
while True:
    
    #Collect List of links
    links = driver.find_element_by_id('lista').find_elements_by_css_selector('a')      
    
    linkLst=[]
    for link in links:
      if link.get_attribute('href'):
        linkLst.append(link.get_attribute('href'))

    #Collect the HTML Table to DataFrame
    tbl = driver.find_element_by_id('lista').get_attribute('outerHTML')
    df = pd.DataFrame()
    df  = pd.read_html(tbl)[0]                                                       

    #Append the Links to DF
    if len(linkLst) > 0:
      df['Links']=linkLst
    
    if dfTotal.empty:
      dfTotal=df
    else:
      dfTotal=pd.concat([dfTotal,df], ignore_index=True, sort =False)
    
    i=i+1
    print("Page: " + str(i) + " Total Rows: " + str(len(dfTotal)) + " Duplicated Rows:" + str(sum(dfTotal.duplicated())))

    next_page_btn = driver.find_element_by_id('lista_next')
    if "disabled" in next_page_btn.get_attribute("class"):
        print("No more pages left")
        break
    else:
        next_page_btn.click() 
        # time.sleep(1)
    
        WebDriverWait(driver, 30).until(
             ajax_complete,  "Timeout waiting for page to load")       


passed
Page: 1 Total Rows: 15 Duplicated Rows:0
Page: 2 Total Rows: 30 Duplicated Rows:0
Page: 3 Total Rows: 45 Duplicated Rows:0
Page: 4 Total Rows: 60 Duplicated Rows:0
Page: 5 Total Rows: 75 Duplicated Rows:0
Page: 6 Total Rows: 90 Duplicated Rows:3
Page: 7 Total Rows: 105 Duplicated Rows:4
Page: 8 Total Rows: 120 Duplicated Rows:4
Page: 9 Total Rows: 135 Duplicated Rows:4
Page: 10 Total Rows: 150 Duplicated Rows:6
Page: 11 Total Rows: 165 Duplicated Rows:6
Page: 12 Total Rows: 180 Duplicated Rows:7
Page: 13 Total Rows: 195 Duplicated Rows:8
Page: 14 Total Rows: 210 Duplicated Rows:9
Page: 15 Total Rows: 225 Duplicated Rows:9
Page: 16 Total Rows: 240 Duplicated Rows:9
Page: 17 Total Rows: 255 Duplicated Rows:9
Page: 18 Total Rows: 270 Duplicated Rows:9
Page: 19 Total Rows: 285 Duplicated Rows:9
Page: 20 Total Rows: 300 Duplicated Rows:11
Page: 21 Total Rows: 315 Duplicated Rows:11
Page: 22 Total Rows: 330 Duplicated Rows:11
Page: 23 Total Rows: 345 Duplicated Rows:11
Page: 24 Total 

In [0]:
''#Verify the values in DFTotal
print(len(dfTotal))
# dfTotal.duplicated()
dfTotalClean=dfTotal.drop_duplicates()
print(len(dfTotalClean))


In [0]:
dfTotalClean.head(3)

,Unnamed: 0,Download,Título,Linha de Atuação,Publicado em,Municípios,Trechos,Links
0,NaN,NaN,Relatório de Avaliação dos Resultados da Gestã...,Avaliação dos Resultados da Gestão,06/11/2019,BRASÍLIA - DF,NaN,https://auditoria.cgu.gov.br/download/13175.pdf
1,NaN,NaN,Relatório de Avaliação dos Resultados da Gestã...,Avaliação dos Resultados da Gestão,05/11/2019,GOIÂNIA - GO,NaN,https://auditoria.cgu.gov.br/download/13543.pdf
2,NaN,NaN,Relatório de Avaliação dos Resultados da Gestã...,Avaliação dos Resultados da Gestão,05/11/2019,APARECIDA DE GOIÂNIA - GO,NaN,https://auditoria.cgu.gov.br/download/13542.pdf


In [0]:
#Take Files From GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Write DF with LINKS values to DISK
dfTotalClean.to_pickle(fileName)